# Επισκευές/Ενισχύσεις - Μέρος 1 (Μαρτ.2021)

In [1]:
from streng.ppp.sections.geometry.rectangular import RectangularSectionGeometry
from streng.ppp.sections.geometry.tee import TeeSectionGeometry
from streng.codes.eurocodes.ec8.cls.seismic_action.spectra import SpectraEc8
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import openseespy.opensees as ops
import openseespy.postprocessing.ops_vis as opsv

## Εισαγωγή αριθμού μητρώου φοιτητών

Αντικαταστήστε τα ψηφία με αυτά του αριθμού μητρώου σας και τρέξτε το jupyter notebook.  
Δε θα χρειαστεί να κάνετε κάποια άλλη αλλαγή ή κάποια εισαγωγή δεδομένων

In [2]:
κ, λ, μ, ν = 9, 8, 7, 6

## Δεδομένα

In [3]:
# Δεδομένα φοιτητών βάσει ΑΕΜ
L = 5.8+0.05*μ
H_is = 3.8+0.05*ν
H_or = 2.8+0.05*ν

if ν<5:
    bc_is=hc_is=0.35
    bc_or=hc_or=0.30
    bw=0.25
    h_is=0.65
    h_or=0.55
else:
    bc_is=hc_is=0.40
    bc_or=hc_or=0.35
    bw=0.25
    h_is=0.70
    h_or=0.60

g = 28.5 + 0.2*κ + 0.5*λ+ 0.4*μ
q = 7.2 + 0.2*κ + 0.2*λ + 0.4*μ
    
cnom = 0.025    
    
fcm = 19+0.4*μ+0.2*ν # σε (MPa)
Ecm = round(22*(fcm/10)**0.3,1)
   
E=2.8*10**7
U = 0.0

if ν<5:
    agR = 0.24
    ductility_class = 'H'
    ground_type = 'B'
    importance = 'III'
    γI=1.2
    q_factor = 4.5 * 1.3
    
else:
    agR = 0.36
    ductility_class = 'M'
    ground_type = 'C'
    importance = 'II' 
    γI=1.0
    q_factor = 3.0 * 1.3

ag = agR*γI

specEC8 = SpectraEc8(αgR=agR,
                     γI=γI,
                     ground_type = ground_type,
                     spectrum_type = 
                     1,
                     η=1.0,
                     q=q_factor,
                     β=0.2)

fy = 430 + 5*ν
fu = 1.25 * fy
fye = 1.15 * fy
fue = 1.10 * fu

### Μάζα

In [4]:
mass = (g+0.3*q)*L*2 / 9.81

## Εμφάνιση αρχικών αποτελεσμάτων (πριν το τρέξιμο της ανάλυσης)

In [5]:
print('Διαστάσεις φορέα')
print(f'L = {L:.2f}m')
print(f'H_ισογ = {H_is:.2f}m')
print(f'H_ορ = {H_or:.2f}m')
print('')
print('Διαστάσεις δοκών')
print(f'Δοκοί ισογείου: b = {bw:.2f}m   h={h_is:.2f}m')
print(f'Δοκοί ορόφων:   b = {bw:.2f}m   h={h_or:.2f}m')
print('')
print('Διαστάσεις στύλων')
print(f'Στύλοι ισογείου: b = {bc_is:.2f}m   h={bc_is:.2f}m')
print(f'Στύλοι ορόφων:   b = {bc_or:.2f}m   h={bc_or:.2f}m')
print('')
print('Υλικά')
print('Σκυρόδεμα')
print(f'fcm = {fcm:.2f}MPa   Ecm={Ecm:.1f}GPa')
print('')
print('Χάλυβας')
print(f'fy = {fy:.2f}MPa')
print(f'fu = {fu:.2f}MPa')
print(f'fye = {fye:.2f}MPa')
print(f'fue = {fue:.2f}MPa')
print('')
print('Φορτία δοκών')
print(f'g = {g:.2f}kN/m   q={q:.2f}kN/m')
print('')
print('Μάζες')
print(f'm = {mass:.2f}t (ίση σε όλους τους ορόφους)')
print('')
# print('Δεδομένα φάσματος σχεδιασμού')
# print(f'γI = {γI:.2f} για κατηγορία σπουδαιότητας {importance}')
# print(f'agR = {agR:.3f}g = {agR*9.81:.3f}m/sec2')
# print(f'ag = {ag:.3f}g = {ag*9.81:.3f}m/sec2')
# print(f'Κατηγορία πλαστιμότητας = DC{ductility_class}')
# print(f'Κατηγορία εδάφους: {ground_type}')
# print(f'TB = {specEC8.TB:.2f}sec')
# print(f'TC = {specEC8.TC:.2f}sec')
# print(f'TD = {specEC8.TD:.2f}sec')
# print(f'S = {specEC8.S:.2f}sec')
# print(f'Συντελεστής συμπεριφοράς q = {specEC8.q:.2f}')


Διαστάσεις φορέα
L = 6.15m
H_ισογ = 4.10m
H_ορ = 3.10m

Διαστάσεις δοκών
Δοκοί ισογείου: b = 0.25m   h=0.70m
Δοκοί ορόφων:   b = 0.25m   h=0.60m

Διαστάσεις στύλων
Στύλοι ισογείου: b = 0.40m   h=0.40m
Στύλοι ορόφων:   b = 0.35m   h=0.35m

Υλικά
Σκυρόδεμα
fcm = 23.00MPa   Ecm=28.2GPa

Χάλυβας
fy = 460.00MPa
fu = 575.00MPa
fye = 529.00MPa
fue = 632.50MPa

Φορτία δοκών
g = 37.10kN/m   q=13.40kN/m

Μάζες
m = 51.56t (ίση σε όλους τους ορόφους)



## Αρχικό τρέξιμο - Υπολογισμός ιδιομορφών/ιδιοπεριόδων

### Διατομές

In [6]:
beam_is = RectangularSectionGeometry(b=bw, h=h_is)
A_beam_is = beam_is.area
Iz_beam_is = beam_is.moment_of_inertia_xx * 0.5
Avy_beam_is = beam_is.shear_area_2 * 0.5

beam_or = RectangularSectionGeometry(b=bw, h=h_or)
A_beam_or = beam_or.area
Iz_beam_or = beam_or.moment_of_inertia_xx * 0.5
Avy_beam_or = beam_or.shear_area_2 * 0.5

col_is = RectangularSectionGeometry(b=bc_is, h=hc_is)
A_col_is = col_is.area
Iz_col_is = col_is.moment_of_inertia_xx * 0.5
Avy_col_is = col_is.shear_area_2 * 0.5

col_or = RectangularSectionGeometry(b=bc_or, h=hc_or)
A_col_or = col_or.area
Iz_col_or = col_or.moment_of_inertia_xx * 0.5
Avy_col_or = col_or.shear_area_2 * 0.5


### Ανάλυση στο OpenSees για υπολογισμό ιδιομορφών

In [7]:
ops.wipe()
ops.model('basic', '-ndm', 2, '-ndf', 3)

E = Ecm * 10**6
U = 0.2
G = E / (2.0*(1.0+U))

ops.node(1, 0., 0.)
ops.node(2, 0., H_is)
ops.node(3, 0., H_is+H_or)
ops.node(4, 0., H_is+2*H_or)
ops.node(5, L, 0.)
ops.node(6, L, H_is)
ops.node(7, L, H_is+H_or)
ops.node(8, L, H_is+2*H_or)
ops.node(9, 2*L, 0.)
ops.node(10, 2*L, H_is)
ops.node(11, 2*L, H_is+H_or)
ops.node(12, 2*L, H_is+2*H_or)

ops.fix(1, 1, 1, 1)
ops.fix(5, 1, 1, 1)
ops.fix(9, 1, 1, 1)

ops.geomTransf('Linear', 1)

ops.mass(int(2), mass, 1.0e-10, 1.0e-10)
ops.mass(int(3), mass, 1.0e-10, 1.0e-10)
ops.mass(int(4), mass, 1.0e-10, 1.0e-10)

for node in [6, 10]:
    ops.equalDOF(2, node, 1)
for node in [7, 11]:
    ops.equalDOF(3, node, 1)
for node in [8, 12]:
    ops.equalDOF(4, node, 1)

# Στύλοι             
ops.element('ElasticTimoshenkoBeam', 1, 1, 2, E, G, A_col_is, Iz_col_is, Avy_col_is, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 2, 2, 3, E, G, A_col_or, Iz_col_or, Avy_col_or, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 3, 3, 4, E, G, A_col_or, Iz_col_or, Avy_col_or, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 4, 5, 6, E, G, A_col_is, Iz_col_is, Avy_col_is, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 5, 6, 7, E, G, A_col_or, Iz_col_or, Avy_col_or, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 6, 7, 8, E, G, A_col_or, Iz_col_or, Avy_col_or, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 7, 9, 10, E, G, A_col_is, Iz_col_is, Avy_col_is, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 8, 10, 11, E, G, A_col_or, Iz_col_or, Avy_col_or, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 9, 11, 12, E, G, A_col_or, Iz_col_or, Avy_col_or, 1)  # , '-mass', 0., '-lMass')
# Δοκοί
ops.element('ElasticTimoshenkoBeam', 10, 2, 6, E, G, A_beam_is, Iz_beam_is, Avy_beam_is, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 11, 3, 7, E, G, A_beam_or, Iz_beam_or, Avy_beam_or, 1)  # , '-mass', 0., '-lMass')  
ops.element('ElasticTimoshenkoBeam', 12, 4, 8, E, G, A_beam_or, Iz_beam_or, Avy_beam_or, 1)  # , '-mass', 0., '-lMass')  
ops.element('ElasticTimoshenkoBeam', 13, 6, 10, E, G, A_beam_is, Iz_beam_is, Avy_beam_is, 1)  # , '-mass', 0., '-lMass')
ops.element('ElasticTimoshenkoBeam', 14, 7, 11, E, G, A_beam_or, Iz_beam_or, Avy_beam_or, 1)  # , '-mass', 0., '-lMass')  
ops.element('ElasticTimoshenkoBeam', 15, 8, 12, E, G, A_beam_or, Iz_beam_or, Avy_beam_or, 1)  # , '-mass', 0., '-lMass')   
             
elem_type = {1: 'col',
             2: 'col',
             3: 'col',
             4: 'col',
             5: 'col',
             6: 'col',
             7: 'col',
             8: 'col',
             9: 'col',
             10: 'beam',
             11: 'beam',
             12: 'beam',
             13: 'beam',
             14: 'beam',
             15: 'beam'}

numEigen = 1
eigenValues = ops.eigen('-fullGenLapack', numEigen)

_periods = []
for i in range(0, numEigen):
    lamb = eigenValues[i]
    period = 2 * np.pi / np.sqrt(lamb)
    _periods.append(period)
    
    eigen_vector = []
    for mn in [2, 3, 4]: # self.model.master_nodes:
        # print(f'eigen {i} mn {mn}')
        eigen_vector.append(ops.nodeEigenvector(mn, i+1, 1))

results_ele = []

### Αποτελέσματα 1ης ιδιομοφής

In [8]:
print(f'Θεμελιώδης ιδιοπερίοδος T{i+1} = {period:.3f}s')
print('Συνιστώσες 1ης ιδιομορφής (δε θα δείτε ακριβώς τα ίδια νούμερα, θα έχετε όμως τις ίδιες αναλογίες):')
print([f'{e:.4f}' for e in eigen_vector])

Θεμελιώδης ιδιοπερίοδος T1 = 0.934s
Συνιστώσες 1ης ιδιομορφής (δε θα δείτε ακριβώς τα ίδια νούμερα, θα έχετε όμως τις ίδιες αναλογίες):
['0.4389', '0.7865', '1.0000']


## Υπολογισμός σεισμικών δράσεων για τέμνουσα βάσης 100kN

In [9]:
# Sd_T = specEC8.Sd(period) * 9.81
M = mass * 3.

# if period <= 2*specEC8.TC:
#     λ_ec8 = 0.85
# else:
#     λ_ec8 = 1.0
    
# Fb = M * Sd_T * λ_ec8
Fb = 100

eigen_vector_array = np.array(eigen_vector)
mass_array = np.array([mass, mass, mass])

In [10]:
# print(f'Επιτάχυνση σχεδιασμού Sd(T) = {Sd_T:.3f}m/sec2')
print(f'Μάζα για το σύνολο του κτιρίου M = {M:.2f}t')
# print(f'λ = {λ_ec8}')
print(f'Τέμνουσα βάσης Fb = {Fb:.2f}kN')

Μάζα για το σύνολο του κτιρίου M = 154.67t
Τέμνουσα βάσης Fb = 100.00kN


### Ιδιομορφικό μοντέλο φόρτισης

In [11]:
Fi_modal = (eigen_vector_array * mass_array) * Fb /sum(eigen_vector_array * mass_array)

In [12]:
for _i, _fi in enumerate(Fi_modal):
    print(f'F{_i+1} = {_fi:.2f}kN')

F1 = 19.72kN
F2 = 35.34kN
F3 = 44.94kN


### Ομοιόμορφο μοντέλο φόρτισης

In [13]:
Fi_uni = (mass_array) * Fb /sum(mass_array)

In [14]:
for _i, _fi in enumerate(Fi_uni):
    print(f'F{_i+1} = {_fi:.2f}kN')

F1 = 33.33kN
F2 = 33.33kN
F3 = 33.33kN


## Υπολογισμός στοχευόμενης μετακίνησης

In [15]:
C0 = 1.3
C1 = 1.0
if period < 0.1:
    C2 = 1.3
elif period > specEC8.TC:
    C2 = 1.1
else:
    interp = interp1d([0.1, specEC8.TC], [1.3, 1.1])
    C2 = interp(period)

C3 = 1.0

Te = period
Se = specEC8.Se(period) * 9.81

δt = C0*C1*C2*C3*Te**2/(4*np.pi**2)*Se

In [16]:
print(f'C0 = {C0:.2f}')
print(f'C1 = {C1:.2f}')
print(f'C2 = {C2:.2f}')
print(f'C3 = {C3:.2f}')
print(f'Te = {Te:.3f}sec')
print(f'Se = {Se:.3f}m/sec2')
print(f'δt = {δt:.3f}m')

C0 = 1.30
C1 = 1.00
C2 = 1.10
C3 = 1.00
Te = 0.934sec
Se = 6.519m/sec2
δt = 0.206m
